In [1]:
from bs4 import BeautifulSoup
import os.path
import os
import numpy as np
import requests
import pandas as pd
import re
import time
import random
from tqdm.notebook import tqdm

## Т.к Циан не самый простой сайт для парсинга, использовала headers - для того, что бы отправить заголовки на сервер в запросе и выглядить более похожим на обычного пользователя, плюс использовала паузы sleep в запросах

In [13]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2171.95 Safari/537.36'}
COLUMNS = {'Название', 'Комнаты', 'Адрес', 'Метро', 'Время до метро', 'Цена', 
           'Цена за квадрат', 'Общая', 'Жилая', 'Кухня', 'Этаж', 'Год постройки', 
           'Строительная серия', 'Тип дома', 'Тип перекрытий', 'Подъезды', 'Лифты', 'Отопление', 'Аварийность', 'Парковка'}

HEADER = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36" ,
    'referer':'https://www.google.com/'
}
#Прописываем функцию, которые проверяет наши спаршеные поля и и приводит их к нужному виду
def check_field(field_name, data):
    if not data: return
    
    if field_name == 'Адрес':
        return data.text.replace('На карте', '')
    elif field_name == 'Цена':
        return data.text[:-1].replace('\xa0', '')
    elif field_name == 'Цена за квадрат':
        return data.text[:-5].replace('\xa0', '')
    else:
        return data.text

#Функция генерирующая страницы ссылки
def generate_cian_url(page_number):
    return f'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={page_number}&region=1'

#функция, которая парсит информацию с одной страницы, проверяет, если выпадает капча
def parse_flat_url(page_number):
    url = generate_cian_url(page_number)
    soup = BeautifulSoup(requests.get(url, headers=HEADER).text)
    divs = soup.find_all('div', class_="_93444fe79c--content--lXy9G")
    
    if divs == []:
        print('Capcha required!')
        return
    
    result = set()
    for item in divs:
        link = item.find('a', class_="_93444fe79c--link--eoxce").get('href')
        result.add(link)
    return result

#функция, которая парсит информацию со всех страниц, которые мы укажем, проверяет, собирает данные в список
def parse_all_urls(page_numbers):
    result = []
    for i in range(1, page_numbers + 1):
        time.sleep(2)
        flat_urls = parse_flat_url(i)
        if not flat_urls:
            break
        result.extend(flat_urls)
    return result


#Собираем данные с одной страницы квартиры, находим нужные нам заголовки и собираем в словарь
def parse_flat(_flat_url):
    result = dict()
    soup = BeautifulSoup(requests.get(_flat_url, headers=HEADER).text)
    name = soup.find('h1', class_= re.compile('\.*--title--\.*'))
    rooms = soup.find('div', class_= re.compile('\.*--title--\.*'))
    address = soup.find('address', class_=re.compile('\.*--address--\.*'))
    metro = soup.find('a', class_=re.compile('\.*--underground_link--\.*'))
    time_to_metro = soup.find('span', class_=re.compile('\.*--underground_time--\.*'))
    price = soup.find('span', class_=re.compile('\.*--price_value--\.*'))
    square_price = soup.find('div', class_=re.compile('\.*--price_per_meter--\.*'))
    
    #на страницах есть поле с информацией, забираем все, что найдем
    info = soup.find_all('div', class_=re.compile('\.*--info--\.*'))
    info_fields = []
    for div in info:
        value = div.find('div', class_=re.compile('\.*--info-value--\.*'))
        field_name = div.find('div', class_=re.compile('\.*--info-title--\.*'))
        info_fields.append((field_name, value))
    
    item = soup.find_all('div', class_=re.compile('\.*--item--\.*'))
    item_fields = []
    for div in item:
        value = div.find('div', class_=re.compile('\.*--value--\.*'))
        field_name = div.find('div', class_=re.compile('\.*--name--\.*'))
        item_fields.append((field_name, value))
    
    result['Название'] = check_field('Название', name)
    result['Комнаты'] = check_field('Комнаты', rooms)
    result['Адрес'] = check_field('Адрес', address)
    result['Метро'] = check_field('Метро', metro)
    result['Время до метро'] = check_field('Время до метро', time_to_metro)
    result['Цена'] = check_field('Цена', price)
    result['Цена за квадрат'] = check_field('Цена за квадрат', square_price)
    
    for field_name, value in info_fields:
        if field_name:
            result[field_name.text] = check_field(field_name, value)
    for field_name, value in item_fields:
        if field_name:
            result[field_name.text] = check_field(field_name, value)
    
    return result
    
#собираем информацию со всех страниц и сохраняем файл с данными
def parse_flats(flat_urls, save_path):
    target_df = pd.DataFrame(columns=COLUMNS)
    target_df.to_csv(save_path, encoding='utf-8', index=False)
    for url in tqdm(flat_urls):
        time.sleep(3)
        flat_fields = parse_flat(url)
        if flat_fields:
            result_df = pd.concat([pd.read_csv(save_path), pd.DataFrame([flat_fields])])
            result_df.to_csv(save_path, index=False)

In [14]:
flat_urls = parse_all_urls(150)

In [15]:
parse_flats(flat_urls, './result2.csv')

  0%|          | 0/4197 [00:00<?, ?it/s]

In [21]:
pd.read_csv('./result2.csv', sep=';')

,Цена за квадрат,Тип дома,Кухня,Лифты,Название,Строительная серия,Адрес,Этаж,Комнаты,Отопление,...,Метро,Жилая,Время до метро,Цена,Аварийность,Срок сдачи,Сдан,Построен,Мусоропровод,Газоснабжение
0,752 522,NaN,NaN,NaN,"3-комн. квартира, 99,52 м²",NaN,"Москва, ЗАО, р-н Дорогомилово, Виктори Парк Ре...",9 из 10,Чистовая,NaN,...,Парк Победы,NaN,⋅ 13 мин. пешком,74890993.0,NaN,4 кв. 2023,NaN,NaN,NaN,NaN
1,279 208,NaN,"22,8 м²",NaN,"2-комн. квартира, 60,6 м²",NaN,"Москва, САО, р-н Дмитровский, Лобненская ул., ...",4 из 16,История цены,NaN,...,Лианозово,"23,8 м²",⋅ 5 мин. на транспорте,16920000.0,NaN,2022,NaN,NaN,NaN,NaN
2,2 117 647,NaN,NaN,NaN,"3-комн. квартира, 139,9 м²",NaN,"Москва, ЦАО, р-н Арбат, ул. Арбат, 39",5 из 5,Без отделки,NaN,...,Смоленская,NaN,⋅ 6 мин. пешком,296258823.0,NaN,NaN,2019.0,NaN,NaN,NaN
3,548 263,NaN,NaN,NaN,"4-комн. квартира, 129,5 м²",NaN,"Москва, ЗАО, р-н Раменки, Минская ул.",3 из 14,Дополнительные предложения по продаже,NaN,...,Минская,NaN,⋅ 14 мин. пешком,71000000.0,NaN,2 кв. 2023,NaN,NaN,NaN,NaN
4,445 920,NaN,NaN,NaN,"2-комн. квартира, 52,7 м²",NaN,"Москва, САО, р-н Левобережный, Акватория жилой...",6 из 20,Чистовая,NaN,...,Беломорская,NaN,⋅ 20 мин. пешком,23500000.0,NaN,2 кв. 2023,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501,215 778,NaN,10 м²,"1 пасс., 1 груз.","1-комн. квартира, 46,9 м²",NaN,"Москва, ЗелАО, р-н Крюково, Зеленоград, мкр. 1...",9 из 19,Помощь риелтора,Индивидуальный тепловой пункт,...,Нахабино,"24,5 м²",⋅ 50 мин. на транспорте,10120000.0,Нет,NaN,NaN,2018.0,Нет,NaN
1502,300 905,Панельный,6 м²,Есть,"2-комн. квартира, 44,2 м²",II-49,"Москва, САО, р-н Головинский, Фестивальная ул....",4 из 9,Помощь риелтора,Центральное,...,Беломорская,"27,7 м²",⋅ 3 мин. на транспорте,13300000.0,Нет,NaN,NaN,1969.0,Есть,Центральное
1503,290 933,NaN,6 м²,NaN,"Студия, 24,6 м²",NaN,"Москва, ЮЗАО, р-н Южное Бутово, Южные Сады жил...",22 из 23,Чистовая,NaN,...,Бунинская аллея,9 м²,⋅ 15 мин. пешком,7156964.0,NaN,4 кв. 2023,NaN,NaN,NaN,NaN
1504,249 128,NaN,"8,5 м²","1 пасс., 1 груз.","1-комн. квартира, 34,4 м²",NaN,"Москва, ЗелАО, р-н Крюково, Зеленоград, мкр. 1...",5 из 11,Помощь риелтора,Индивидуальный тепловой пункт,...,Опалиха,"17,9 м²",⋅ 50 мин. на транспорте,8570000.0,Нет,NaN,NaN,2018.0,Есть,NaN
